# Multi-agent Systems

Multi-agent systems solve two fundamental problems that emerge when building sophisticated LLM applications: task complexity and context window constraints.

When you build a single monolithic agent to handle complex workflows, you quickly hit limitations. The agent's prompt must contain instructions for all possible scenarios, tool definitions for every action it might take, and examples covering diverse use cases. LLMs perform best when given focused, specific instructions. A 5,000-token system prompt trying to cover everything will naturally produce less accurate results than a 500-token prompt focused on one domain.

Furthermore, it's difficult to debug a complex agent. When something goes wrong, it's hard to isolate whether the issue is in the routing logic, domain knowledge, or tool execution. Everything is entangled.

A multi-agent approach decomposes this into specialized components, each with a narrow focus and minimal context requirements.

In [ ]:
from __future__ import annotations

import sys
from typing import Annotated, Literal

from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain.messages import HumanMessage, ToolMessage
from langchain.tools import ToolRuntime, tool
from langgraph.graph import MessagesState
from langgraph.types import Command
from loguru import logger
from pydantic import BaseModel, Field

from chain_reaction.config import APIKeys, ModelBehavior, ModelName
from chain_reaction.utils import get_last_message, get_structured_response

# Define chat model
api_keys = APIKeys()

chat_model = init_chat_model(
    model=ModelName.CLAUDE_HAIKU,
    timeout=None,
    max_retries=2,
    api_key=api_keys.anthropic,
    **ModelBehavior.deterministic().model_dump(),
)

# Remove the default handler to apply a custom one
logger.remove()

# Add the new handler with the custom format and enable colorization
logger.add(
    sys.stderr,
    colorize=True,
    level="DEBUG",
)

# Reassign the logger with the ansi option set as default
logger = logger.opt(ansi=True)

# Define sub-agents

In [ ]:
class CalculationResult(BaseModel):
    """Response model for calculation results."""

    result: float = Field(description="The result of the calculation.")

    @classmethod
    def get_response(cls, response: dict) -> CalculationResult | None:
        """Extract a CalculationResult instance from the agent response.

        Args:
            response (dict): The agent response containing the structured response.

        Returns:
            CalculationResult | None: The extracted CalculationResult if available, otherwise None.
        """
        return get_structured_response(response, cls)


@tool
def calculate_square_root(number: float) -> float:
    """Calculates the square root of a given number.

    Args:
        number (float): The number to calculate the square root of.

    Returns:
        float: The square root of the number.
    """
    return number**0.5


@tool
def calculate_square(number: float) -> float:
    """Calculates the square of a given number.

    Args:
        number (float): The number to calculate the square of.

    Returns:
        float: The square of the number.
    """
    return number**2


square_root_agent = create_agent(
    model=chat_model,
    tools=[calculate_square_root],
    system_prompt="""
    You're a helpful assistant that can perform square root calculations.
    Use the provided calculation tools to answer user questions accurately.
    """,
    response_format=CalculationResult,
)

square_agent = create_agent(
    model=chat_model,
    tools=[calculate_square],
    system_prompt="""
    You're a helpful assistant that can perform square calculations.
    Use the provided calculation tools to answer user questions accurately.
    """,
    response_format=CalculationResult,
)

In [ ]:
response = square_root_agent.invoke({"messages": [HumanMessage(content="What is the square root of 16?")]})
CalculationResult.get_response(response)

# Define main agent

In [ ]:
@tool
def invoke_square_agent(number: float) -> CalculationResult:
    """Invokes the square agent to calculate the square of a number.

    Args:
        number (float): The number to calculate the square of.

    Returns:
        CalculationResult: The result of the square calculation.
    """
    logger.debug(f"Invoking <magenta>square agent</> for number: {number}")
    response = square_agent.invoke({"messages": [HumanMessage(content=f"What is the square of {number}?")]})
    result = CalculationResult.get_response(response)
    if result is not None:
        return result
    else:
        raise ValueError("Failed to get square calculation result.")


@tool
def invoke_square_root_agent(number: float) -> CalculationResult:
    """Invokes the square root agent to calculate the square root of a number.

    Args:
        number (float): The number to calculate the square root of.

    Returns:
        CalculationResult: The result of the square root calculation.
    """
    logger.debug(f"Invoking <red>square root agent</> for number: {number}")
    response = square_root_agent.invoke({"messages": [HumanMessage(content=f"What is the square root of {number}?")]})
    result = CalculationResult.get_response(response)
    if result is not None:
        return result
    else:
        raise ValueError("Failed to get square calculation result.")


main_agent = create_agent(
    model=chat_model,
    tools=[invoke_square_root_agent, invoke_square_agent],
    system_prompt="""
    You're a helpful assistant that can perform both square and square root calculations.
    Use the provided sub-agents to answer user questions accurately.
    """,
)

In [ ]:
response = main_agent.invoke({"messages": [HumanMessage(content="What is the square of the square root of 81?")]})
last_message = get_last_message(response)
if last_message:
    print(last_message.content)

# Define main agent with state

For complex tasks, your main agent will want to update its internal state to keep track on responses from sub-agents.

## Define another subagent

In [ ]:
type Operation = Literal["add", "subtract"]


@tool
def add_or_subtract(number1: float, number2: float, operation: Operation) -> float:
    """Performs addition or subtraction on two numbers.

    Args:
        number1 (float): The first number.
        number2 (float): The second number.
        operation (Operation): The operation to perform.

    Returns:
        float: The result of the operation.

    Raises:
        ValueError: If an invalid operation is provided.
    """
    if operation == "add":
        return number1 + number2
    elif operation == "subtract":
        return number1 - number2
    else:
        raise ValueError("Invalid operation. Choose 'add' or 'subtract'.")


add_or_subtract_agent = create_agent(
    model=chat_model,
    tools=[add_or_subtract],
    system_prompt="""
    You're a helpful assistant that can perform addition and subtraction calculations.
    Use the provided calculation tools to answer user questions accurately.
    """,
    response_format=CalculationResult,
)


@tool
def invoke_add_or_subtract(number1: float, number2: float, operation: Operation) -> CalculationResult:
    """Invokes the addition or subtraction agent to perform the specified operation.

    Args:
        number1 (float): The first number.
        number2 (float): The second number.
        operation (Operation): The operation to perform.

    Returns:
        CalculationResult: The result of the calculation.

    Raises:
        ValueError: If the calculation result could not be obtained.
    """
    logger.debug(
        f"Invoking <yellow>add or subtract agent</> for numbers: {number1}, {number2} with operation: {operation}"
    )
    response = add_or_subtract_agent.invoke({
        "messages": [HumanMessage(content=f"What is the result of {number1} {operation} {number2}?")]
    })
    result = CalculationResult.get_response(response)
    if result is not None:
        return result
    else:
        raise ValueError("Failed to get calculation result.")

## Define state schema and tools for tracking calculations

In [ ]:
class CalculationState(MessagesState):
    """Agent state to store calculation results over time.

    Attributes:
        calculation_results (dict[str, CalculationResult]): Stored calculation results as
            {variable_name: CalculationResult} mapping. Uses the merge_dicts reducer to combine calculation results
            from multiple parallel tool calls.
    """

    calculation_results: Annotated[dict[str, CalculationResult], merge_dicts]


def merge_dicts(a: dict, b: dict) -> dict:
    """Merges two dictionaries into a single dictionary.

    Args:
        a (dict): The first dictionary.
        b (dict): The second dictionary.

    Returns:
        dict: The merged dictionary.
    """
    return {**a, **b}


@tool
def save_calculation_result_to_state(
    variable_name: str,
    result: CalculationResult,
    runtime: ToolRuntime[CalculationState],
) -> Command:
    """Updates the calculation state with new results.

    - Store intermediate calculation results from sub-agents for later retrieval.
    - Only store results that might be needed later.

    Args:
        variable_name (str): The name of the variable to store the result under.
        result (CalculationResult): The calculation result to store.
        runtime (ToolRuntime[CalculationState]): The tool runtime with access to the agent state.

    Returns:
        Command: The command to update the agent state.

    Raises:
        RuntimeError: If variable is already present in state.
    """
    logger.debug(f"<green>Saving calculation result</> {result.result} for {variable_name}")

    # Get current results from state
    current_results = runtime.state.get("calculation_results", {})

    # Check if variable already exists
    if variable_name in current_results:
        logger.error(f"Variable {variable_name} already exists in state.")
        raise RuntimeError(f"Variable {variable_name} already exists in state.")

    # Update results
    current_results[variable_name] = result

    # Generate command to update current favorites in state
    return Command(
        update={
            "calculation_results": current_results,
            "messages": [
                ToolMessage(
                    content=f"Updated {variable_name} to {result.result}.",
                    tool_call_id=runtime.tool_call_id,
                )
            ],
        }
    )


@tool
def get_result_from_state(
    variable_name: str,
    runtime: ToolRuntime[CalculationState],
) -> float:
    """Retrieves a calculation result from the agent state using a variable name.

    Args:
        variable_name (str): The name of the variable to retrieve.
        runtime (ToolRuntime[CalculationState]): The tool runtime with access to the agent state.

    Returns:
        float: The retrieved calculation result.

    Raises:
        ValueError: If the calculation result could not be obtained.
    """
    # Get current results from state
    current_results: dict[str, CalculationResult] = runtime.state.get("calculation_results", {})
    result = current_results.get(variable_name)
    if result is None:
        logger.error(f"No result found in state for variable: {variable_name}")
        raise ValueError(f"No result found in state for variable: {variable_name}")
    logger.debug(f"<white>Retrieving calculation result</> {result.result} for {variable_name}")
    return result.result

## Redefine main agent with state

In [ ]:
main_agent = create_agent(
    model=chat_model,
    system_prompt="""
    You're a helpful assistant that can perform addition, subtraction, square, and square root calculations.

    Use the provided sub-agents tools to handle calculations:
    - invoke_add_or_subtract
    - invoke_square_agent
    - invoke_square_root_agent

    Use the provided state tools for storing and retrieving intermediate results:
    - save_calculation_result_to_state
    - get_result_from_state

    ALWAYS store intermediate results and retrieve them for subsequent calculations; do NOT recalculate values.

    For example, if you asked to calculate x1 - x2, where x1 = sqrt(x), x2 = y**2, x=16, y=3:
    1. First, calculate the square root of x using invoke_square_root_agent.
    2. Store the result with save_calculation_result_to_state under the variable name "x1".
    3. Next, calculate the square of y using invoke_square_agent.
    4. Store the result with save_calculation_result_to_state under the variable name "x2".
    5. Retrieve the value of "x1" using get_result_from_state.
    6. Retrieve the value of "x2" using get_result_from_state.
    7. Finally, perform the subtraction using invoke_add_or_subtract.

    Put your final calculation result in a CalculationResult response format and return it.
    DO NOT store the final result.
    STOP once you have provided the final result.
    """,
    tools=[
        invoke_add_or_subtract,
        invoke_square_agent,
        invoke_square_root_agent,
        get_result_from_state,
        save_calculation_result_to_state,
    ],
    state_schema=CalculationState,
    response_format=CalculationResult,
)

In [ ]:
response = main_agent.invoke(
    {
        "messages": [
            HumanMessage(
                "What is the value of x1**x4 - x3 + x2, given that: "
                "x=9 and y=16. "
                "x1=x**2 - sqrt(y). "
                "x2=x1**2 + sqrt(y). "
                "x3=sqrt(x2) + 4. "
                "x4=x3 - x1."
            )
        ]
    },
)
last_message = get_last_message(response)
if last_message:
    print(last_message.content)

In [ ]:
response